![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Prompt Lab on Notebook v1.0

이 노트북에는 프롬프트 추론을 시연하는 단계와 코드가 포함되어 있습니다.  
프롬프트 추론을 시연하는 단계와 코드가 포함되어 있습니다. 여기에는 API 키를 사용하여 인증하는 Python API 명령  
을 사용한 인증과 WML API를 사용한 프롬프트 추론에 대해 소개합니다.  
**Note:** 프롬프트 랩을 사용하여 생성된 노트북 코드는 성공적으로 실행됩니다.
코드가 수정되거나 순서가 바뀌면 성공적으로 실행된다는 보장은 없습니다.  
자세한 내용은 다음을 참조하세요: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">프롬프트 랩에서 작업을 노트북으로 저장하기.</a>

Python에 어느 정도 익숙하면 도움이 됩니다. 이 노트북은 Python 3.10을 사용합니다.

## Notebook goals
이 노트북의 학습 목표는 다음과 같습니다:

* IBM Cloud 개인 API 키에서 자격 증명을 얻기 위한 Python 함수 정의하기
* 모델 객체의 매개변수 정의하기
* 모델 개체를 사용하여 정의된 모델 ID, 매개변수 및 프롬프트 입력을 사용하여 응답 생성하기

## 소개
watson studio에서 실습하는 것을 vscode에서 실습하기 위해 lab입니다.  


#### 실습환경 구성

In [ ]:
!pip install ibm-watson-machine-learning python-dotenv

In [ ]:
from dotenv  import load_dotenv
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.model import Model 

import os 

load_dotenv()

credentials = {
		"url" : "https://us-south.ml.cloud.ibm.com",
		"apikey" : os.getenv("API_KEY", None)
	}

project_id = os.getenv("PROJECT_ID", None)

def get_completion(prompt):
    # Instantiate parameters for text generation
    params = {
        GenParams.DECODING_METHOD: 'greedy',
        GenParams.MIN_NEW_TOKENS: 1,
        GenParams.MAX_NEW_TOKENS: 400,
        GenParams.TEMPERATURE: 0.2,
        GenParams.RANDOM_SEED: 42,
        GenParams.REPETITION_PENALTY: 1.0,
        GenParams.STOP_SEQUENCES : ["\n\n"]
    }
    
    # Instantiate a model proxy object to send your requests
    model = Model(
        model_id='meta-llama/llama-3-70b-instruct',
        params=params,
        credentials=credentials, # type: ignore
        project_id=project_id)

    response = model.generate_text(prompt = prompt)

    return response

#### 실습 예제

In [ ]:
text_1 = """
```차 한잔 끓이는 방법은 간단해요! 먼저 물을 좀 끓여야 합니다. 
그 동안 컵을 들고 그 안에 티백을 넣으세요. 
물이 충분히 뜨거워지면 티백 위에 부어주세요. 
차가 가파르게 될 수 있도록 잠시 그대로 두십시오. 
몇 분 후 티백을 꺼냅니다. 
원하시면 설탕이나 우유를 조금 첨가해 드셔도 좋습니다. 
그리고 그게 다입니다.! 맛있는 차 한잔을 즐기실 수 있습니다.```
"""
prompt = f"""

{text_1}

삼중따옴표로 구분된 텍스트가 제공됩니다. 일련의 지침이 포함된 경우 해당 지침을 다음 형식으로 다시 작성하세요.
단계 1 - ...
단계 2 - …
…
Step N - …

결과 : 
"""

response = get_completion(prompt)
print(response)

### 원칙 1: 명확하고 구체적인 지침을 작성하세요.

모델이 의도와 원하는 결과를 이해할 수 있도록 프롬프트가 명확하고 간결해야 합니다. 다양한 해석으로 이어질 수 있는 모호한 언어나 표현을 피하세요. 이는 다음과 같은 전략을 통해 달성할 수 있습니다.

#### 전략 1: 구분 기호를 사용하여 입력의 개별 부분을 명확하게 나타냅니다.

구분 기호는 잘못된 사용자 입력으로 인한 잠재적인 간섭을 방지하는 데 도움이 됩니다. 구분 기호의 예는 다음과 같습니다.

- Triple quotes: `"""`
- Triple backticks: ```` ``` ````
- Triple dashes: `---`
- Angle brackets: `<>`
- XML tags: `<tag>`

Example:

```python
text = """
```가능한 한 명확하고 구체적인 지침을 제공하여 모델이 수행하기를 원하는 작업을 표현해야 합니다. 
이렇게 하면 모델이 원하는 출력으로 안내되고 관련이 없거나 잘못된 응답을 받을 가능성이 줄어듭니다. 
명확한 프롬프트 작성과 짧은 프롬프트 작성을 혼동하지 마십시오. 
대부분의 경우 프롬프트가 길수록 모델에 대한 명확성과 맥락이 더 높아져 더 자세하고 관련성이 높은 결과를 얻을 수 있습니다.```
"""
prompt = f"""

{text}

삼중따옴표로 구분된 텍스트를 한 문장으로 요약합니다.

결과 : 
"""
response = get_completion(prompt)
print(response)
```

#### 전략 2: 구조화된 출력 요청

이 접근 방식은 Python 프로그램에서 읽고 사전 형식으로 변환할 수 있는 JSON 출력과 같은 모델 출력을 프로그램에 직접 사용할 수 있도록 하는 데 도움이 됩니다.

Example:

```python
prompt = """
저자 및 장르와 함께 구성된 책 제목 3개 목록을 생성합니다. book_id, 제목, 저자, 장르 키를 사용하여 JSON 형식으로 제공하세요.
"""
response = get_completion(prompt)
print(response)
```

<mark> Result:
```json
[
  {
    "book_id": 1,
    "title": "The Lost City of Zorath",
    "author": "Aria Blackwood",
    "genre": "Fantasy"
  },
  {
    "book_id": 2,
    "title": "The Last Survivors",
    "author": "Ethan Stone",
    "genre": "Science Fiction"
  },
]

```

#### 전략 3: 원하는 조건을 주어, 충족 여부를 확인

작업 완료에 충족되어야 하는 전제 조건이 있는 경우 모델에 이러한 조건을 먼저 확인하고 충족되지 않으면 시도를 중지하도록 지시해야 합니다.

Example:

```python
text_1 = """
```차 한잔 끓이는 방법은 간단해요! 먼저 물을 좀 끓여야 합니다. 
그 동안 컵을 들고 그 안에 티백을 넣으세요. 
물이 충분히 뜨거워지면 티백 위에 부어주세요. 
차가 가파르게 될 수 있도록 잠시 그대로 두십시오. 
몇 분 후 티백을 꺼냅니다. 
원하시면 설탕이나 우유를 조금 첨가해 드셔도 좋습니다. 
그리고 그게 다입니다.! 맛있는 차 한잔을 즐기실 수 있습니다.```
"""
prompt = f"""

{text_1}

삼중따옴표로 구분된 텍스트가 제공됩니다. 일련의 지침이 포함된 경우 해당 지침을 다음 형식으로 다시 작성하세요.
단계 1 - ...
단계 2 - …
…
Step N - …

결과 : 
"""
response = get_completion(prompt)
print(response)
```

<mark> Output:
```
1 단계 - 물을 좀 끓여야 합니다.
2 단계 - 컵을 들고 그 안에 티백을 넣으세요.
3 단계 - 물이 충분히 뜨거워지면 티백 위에 부어주세요.
4 단계 - 차가 우러날 수 있도록 잠시 놓아두세요.
5 단계 - 몇 분 후 티백을 꺼내세요..
6 단계 - 원하시면 설탕이나 우유를 조금 첨가해 드셔도 좋습니다.
```

#### 전략 4: “N-shot” Prompting

모델에 하나 이상의 샘플 프롬프트를 제공하면 예상되는 출력을 명확히 하는 데 도움이 됩니다.
Providing the model with one or more sample prompts helps clarify the expected output.
Zero-shot 및 Few-shot 프롬프트를 포함한 N-shot 프롬프트는 예측을 위해 모델에 제공되는 "훈련" 사례 또는 단서의 수를 나타냅니다.

<table width="80%">
  <tr>
    <td width="20%" style="text-align: center">유형</td>
    <td width="40%" style="text-align: center">설명</td>
    <td width="40%" style="text-align: center"> 예</td>
  </tr>
  <tr>
    <td style="text-align: left;">Zero-shot 프롬프팅</td>
    <td style="text-align: left;">모델은 추가 교육 없이 예측을 수행합니다. 이는 분류 또는 텍스트 변환과 같은 일반적인 간단한 문제에 효과적입니다..</td>
    <td style="text-align: left;">분류 (예. 감정분석, 스팸 분류 => "이 메일은 스펨인가요? 아닌가요? ==> <i>이메일 콘텐츠 붙여넣기</i> "), 텍스트 변환(예: 번역, 요약, 확장) 및 LLM이 주로 교육받은 간단한 텍스트 생성</td>
  </tr>
  <tr>
    <td style="text-align: left;">Few-shot 프롬프팅</td>
    <td style="text-align: left;">이러한 작은 예를 기반으로 출력을 조정하기 위해 소량의 데이터(일반적으로 2~5개)를 사용합니다. 이러한 예는 보다 상황에 맞는 문제에 대해 더 나은 성능을 발휘하도록 모델을 조정하기 위한 것입니다.</td>
    <td style="text-align: left;">"다음 영어 문장을 프랑스어로 번역하세요: '안녕하세요, 잘 지내세요?' '나는 괜찮아요, 고마워요.'" </td>
  </tr>    
     
</table>    

### 원칙 2: 모델에게 "생각"할 시간을 주십시오

이 원칙은 사고 사슬의 아이디어를 활용하여 복잡한 작업을 N개의 순차적 하위 작업으로 나누어 모델이 단계별로 사고하고 보다 정확한 출력을 생성할 수 있도록 합니다.

#### 전략 1: 작업을 완료하는 데 필요한 단계 지정

다음은 텍스트를 요약하고, 프랑스어로 번역하고, 프랑스어 요약에 이름을 나열하고, 마지막으로 데이터를 JSON 형식으로 출력하는 것과 관련된 예입니다. 필요한 단계를 제공함으로써 모델은 이전 단계의 결과를 참조하고 출력의 정확성을 향상시킬 수 있습니다.


#### 전략 2: 성급하게 결론을 내리기 전에 모델에 자체 솔루션을 마련하도록 지시

작업이 너무 복잡하거나 설명이 너무 적으면 모델은 추측을 통해서만 결론을 도출할 수 있습니다. 따라서 이 경우 문제에 대해 생각하는 데 더 오랜 시간이 걸리도록 모델에 지시할 수 있습니다.

## Prompting on  IBM Watsonx
ibm watsonx platform에서 prompting을 하기위해 사전에 설치해야하는 파이썬 패키지를 다음과 같이 설치합니다. 

In [ ]:
!pip install ibm-watson-machine-learning python-dotenv

In [ ]:
from dotenv import load_dotenv
import os  

load_dotenv()

credentials = {
		"url" : "https://us-south.ml.cloud.ibm.com",
		"apikey" : os.getenv("API_KEY", None)
	}

In [ ]:
project_id = os.getenv("PROJECT_ID", None)

In [ ]:
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

def send_to_watsonxai(prompts,
                    model_name="meta-llama/llama-2-70b-chat",
                    decoding_method="greedy",
                    max_new_tokens=100,
                    min_new_tokens=30,
                    temperature=1.0,
                    repetition_penalty=1.0,
                    stop_sequence=['\n\n']
                    ):
    '''
   프롬프트 및 매개 변수를 watsonx.ai로 보내기 위한 function
    
    Args:  
        prompts: 텍스트 프롬프트
        decoding_method: "sample" or "greedy"
        max_new_token:int watsonx.ai parameter for max new tokens/response returned
        temperature:float watsonx.ai parameter for temperature (range 0>2)
        repetition_penalty:float watsonx.ai parameter for repetition penalty (range 1.0 to 2.0)

    Returns: None
        prints response
    '''

    assert not any(map(lambda prompt: len(prompt) < 1, prompts)), "make sure none of the prompts in the inputs prompts are empty"

    # Instantiate parameters for text generation
    model_params = {
        GenParams.DECODING_METHOD: decoding_method,
        GenParams.MIN_NEW_TOKENS: min_new_tokens,
        GenParams.MAX_NEW_TOKENS: max_new_tokens,
        GenParams.RANDOM_SEED: 42,
        GenParams.TEMPERATURE: temperature,
        GenParams.REPETITION_PENALTY: repetition_penalty,
        GenParams.STOP_SEQUENCES : stop_sequence
    }
    
    # Instantiate a model proxy object to send your requests
    model = Model(
        model_id=model_name,
        params=model_params,
        credentials=credentials, # type: ignore
        project_id=project_id)

    response = model.generate_text(prompt = prompt)

    return response   

In [4]:
FLAN_T5_XXL = 'google/flan-t5-xxl'
FLAN_UL2 = 'google/flan-ul2'
GPT_NEOX = 'eleutherai/gpt-neox-20b'
GRANITE_13B_CHAT = 'ibm/granite-13b-chat-v1'
GRANITE_13B_INSTRUCT = 'ibm/granite-13b-instruct-v1'
LLAMA_2_70B_CHAT = 'meta-llama/llama-2-70b-chat'
LLAMA_3_70B_INSTRUCT="meta-llama/llama-3-70b-instruct"
MPT_7B_INSTRUCT2 = 'ibm/mpt-7b-instruct2'
MT0_XXL = 'bigscience/mt0-xxl'
STARCODER = 'bigcode/starcoder'
MISTRAL_LARGE = "mistralai/mistral-large"

#### 원칙1 > 구분 기호를 사용하여 입력의 개별 부분을 명확하게 표시 

In [ ]:
input = """

가능한 한 명확하고 구체적인 지침을 제공하여 모델이 수행하기를 원하는 작업을 표현해야 합니다. 
이렇게 하면 모델이 원하는 출력으로 안내되고 관련이 없거나 잘못된 응답을 받을 가능성이 줄어들고,
명확한 프롬프트 작성과 짧은 프롬프트 작성을 혼동하지 마십시오. 
대부분의 경우 프롬프트가 길수록 모델에 대한 명확성과 맥락이 더 높아져 더 자세하고 관련성이 높은 결과를 얻을 수 있습니다.

위 세 개로 구분된 텍스트를 한 문장으로 요약해서 한글로 출력합니다. 
"""

prompt = f"""

{input}
"""

response = send_to_watsonxai(prompts=[prompt], model_name=LLAMA_3_70B_INSTRUCT, decoding_method="greedy", max_new_tokens=400,
                              min_new_tokens=1, temperature=0, repetition_penalty=1.0)
print(response)

#### 원칙1 > 구조화된 출력형식 요청

#### 실습 1

In [ ]:
input = """ 
입력 : 저자 및 장르와 함께 구성된 책 제목 3개 목록을 생성합니다. 
book_id, 제목, 저자, 장르 키를 사용하여 JSON 형식으로 출력해줘 

출력 : 

"""

prompt = f"""

{input}
"""

response = send_to_watsonxai(prompts=[prompt], model_name=LLAMA_3_70B_INSTRUCT, decoding_method="sample", max_new_tokens=200,
                              min_new_tokens=1, temperature=0, repetition_penalty=1.0, stop_sequence=["\n\n", "```"])
print(response)

#### 실습 2 

In [ ]:
input = '''
def generate_sentence(sentence: str) -> str:
    """
    입력된 문장의 감정(긍정 또는 부정)을 예측합니다.
 
    Parameters:
        sentence (str): Input sentence
    
    Returns:
        str: Sentiment of the input ('positive' or 'negative')
    """
    # 다른 곳에 sentiment_is_positive(sentence) 함수가 정의되어 있다고 가정합니다.
    if sentiment_is_positive(sentence):
        return "positive"
    else:
        return "negative"


print(generate_sentence("매력적인 분위기가 느껴집니다"))

result : 
'''

prompt = f"""

{input}
"""

response = send_to_watsonxai(prompts=[prompt], model_name=LLAMA_3_70B_INSTRUCT, decoding_method="greedy", max_new_tokens=400,
                              min_new_tokens=1, temperature=0, repetition_penalty=1.0, stop_sequence=["\n\n", "```"])
print(response)

## 원칙1 > 구분 기호를 사용하여 입력의 개별 부분을 명확하게 표시

In [ ]:
input = """

다음 텍스트에서 장소의 이름을 추출해 줘.

원하는 형식:
Place: <comma_separated_list_of_places>
입력:  이러한 발전은 연구자들에게 격려가 되지만, 
많은 것들이 여전히 수수께끼입니다. 리스본에 있는 샴팔리마우드 센터(CCU: Champalimaud Centre for the Unknown)의 신경면역학자인 Henrique Veiga-Fernandes는
"뇌와 주변부에서 보이는 효과 사이에 블랙박스가 있는 경우가 종종 있습니다."라고 말합니다. 그리고 다음과 같이 덧붙입니다. 
"치료적 맥락에서 이를 사용하고자 할 경우, 그 메커니즘을 실제로 이해할 필요가 있습니다."
"""

prompt = f"""

{input}
"""

response = send_to_watsonxai(prompts=[prompt], model_name=LLAMA_3_70B_INSTRUCT, decoding_method="greedy", max_new_tokens=400,
                              min_new_tokens=1, temperature=1, repetition_penalty=1.0, stop_sequence=["\n\n", "```"])

print(response)

#### 원칙1 > 조건이 충족되는지 확인하기 

In [ ]:
input = """
```차 한잔 끓이는 방법은 간단해요! 먼저 물을 좀 끓여야 합니다. 그 동안 컵을 들고 그 안에 티백을 넣으세요. 
물이 충분히 뜨거워지면 티백 위에 부어주세요. 차가 가파르게 될 수 있도록 잠시 그대로 두십시오. 
몇 분 후 티백을 꺼냅니다. 원하시면 설탕이나 우유를 조금 첨가해 드셔도 좋습니다. 
그리고 그게 다입니다.! 맛있는 차 한잔을 즐기실 수 있습니다.```
"""

prompt = f"""

{input}

삼중따옴표로 구분된 텍스트가 제공됩니다. 일련의 지침이 포함된 경우 해당 지침을 다음 형식으로 다시 작성하세요.
단계 1 - ...
단계 2 - …
…
단계 N - …

결과 : 
"""

response = send_to_watsonxai(prompts=[prompt], model_name=LLAMA_3_70B_INSTRUCT, decoding_method="greedy", max_new_tokens=200,
                              min_new_tokens=1, temperature=1, repetition_penalty=1.0)

print(response)

#### 원칙1 > N-shot Prompting 

#### 실습1- 프롬프트로 삼행시를 작성해 보세요

In [ ]:
input = """
Context : 입력한 값을 한글로 삼행시를 작성.
예제 : 
입력 : 아이유를 삼행시로 만들어줘.
출력 : 
아: 아름다운 
이: 이별을 
유: 유쾌하게 노래하는 아이유

입력 :  IBM을 삼행시로 만들어줘.
출력 :

"""

prompt = f"""

{input}
"""

response = send_to_watsonxai(prompts=[prompt], model_name=LLAMA_3_70B_INSTRUCT, decoding_method="greedy", max_new_tokens=400,
                              min_new_tokens=1, temperature=1, repetition_penalty=1.0)

print(response)

#### 과제- 프롬프트로 사행시를 작성해 보세요

In [ ]:
input = """
Context : 입력한 값을 한글로 삼행시를 작성.

입력 : 블랙핑크를 사행시를 만들어줘 
출력 : 

"""

prompt = f"""

{input}
"""

response = send_to_watsonxai(prompts=[prompt], model_name=LLAMA_3_70B_INSTRUCT, decoding_method="greedy", max_new_tokens=400,
                              min_new_tokens=1, temperature=1, repetition_penalty=1.0)

print(response)

#### 실습2 : 2-shot 프롬프트

In [ ]:
input = '''
def generate_sentence(sentence: str) -> str:
    """
    입력된 문장의 감정(긍정 또는 부정)을 예측합니다.
 
    Parameters:
        sentence (str): Input sentence
    
    Returns:
        str: Sentiment of the input ('positive' or 'negative')
    """
    # 다른 곳에 sentiment_is_positive(sentence) 함수가 정의되어 있다고 가정합니다.
    if sentiment_is_positive(sentence):
        return "positive"
    else:
        return "negative"

예시 : 
print(generate_sentence("새 제품이 빈번하게 먹통이 되어 환불을 받고 싶다"))
negative

print(generate_sentence("새 제품을 구매하고, 가성비가 좋아 부모님 것도 구매했어요."))
positive

print(generate_sentence("이 제품은 완전히 쓰레기입니다. 하지만 가격이 싸서 구매했어요. 그래도 사용해보고 싶어요. 그리고 좋은 점도 있어요. 그리고 좋은 점이 더 많아요."))

'''


prompt = f"""

{input}
"""

response = send_to_watsonxai(prompts=[prompt], model_name=LLAMA_3_70B_INSTRUCT, decoding_method="greedy", max_new_tokens=400,
                              min_new_tokens=1, temperature=1, repetition_penalty=1.0)

print(response)

#### CoT 프롬프트 

#### 실습 1

In [ ]:
input = """
질문: 리사는 테니스공 5개를 가지고 있습니다. 
    그는 테니스 공 2캔을 더 삽니다. 각 캔에는 테니스 공 3개가 들어 있습니다. 
    이제 리사는 몇 개의 테니스 공을 가지고 있을까요?
답변 : 
"""

prompt = f"""

{input}
"""

response = send_to_watsonxai(prompts=[prompt], model_name=LLAMA_3_70B_INSTRUCT, decoding_method="greedy", max_new_tokens=400,
                              min_new_tokens=1, temperature=0.2, repetition_penalty=1.0)

print(response)

#### 과제 : 아래 프롬프트를 LLM이 CoT 사고사슬을 하도록 수정하세요.

In [ ]:
input = """ 
Q: 조는 평균적으로 분당 25번의 펀치를 던집니다. 시합은 3분씩 5라운드로 진행됩니다. 조는 몇 번의 펀치를 던졌을까요? 
A: 

"""

prompt = f"""

{input}
"""

response = send_to_watsonxai(prompts=[prompt], model_name=LLAMA_3_70B_INSTRUCT, decoding_method="greedy", max_new_tokens=400,
                              min_new_tokens=1, temperature=0.2, repetition_penalty=1.0)

print(response)